# Reactive timeseries via delay

In addition to combining multiple callback *building blocks*, we can set a delay in an ```AcquisitionTaskGenerator```. Tasks generated by it will be temporally spaced by (at least) that delay.

In [1]:
import logging
import sys

import specpy as sp

from autosted import AcquisitionPipeline
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.stoppingcriteria import MaximumAcquisitionsStoppingCriterion
from autosted.callback_buildingblocks import JSONSettingsLoader

# configure logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [2]:
parameters = sp.get_application().value_at("", sp.ValueTree.Measurement).get()

# Alternatively, load from a file or specify path
# parameters = "path/to/parameters.json"

Here, we build a pipeline and AcquisitionTaskGenerator that just repeats the same measurement. However, by setting ```delay=10```, there will be a 10-second delay between acquisitions (the time in measured for the starts of imaging, so if the scan itself takes longer than 10 seconds, there will be no delay).

In [ ]:
pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test", hierarchy_levels=["image"]
)

# callback to repeatedly image with same settings, with delay
next_image_callback = AcquisitionTaskGenerator(
    "image",
    JSONSettingsLoader(parameters),
    delay=10,  # time to wait from last acquisition at level "image"
)

pipeline.add_callback(next_image_callback, "image")

# stop after 10 images
pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(10))

pipeline.run(next_image_callback)

By attaching a second callback to enqueue "fast" images with lower delay, we can switch imaging speed upon an event that can be detected with user-defined functions (here we just use a dummy function that switches after a certain number of slow images).

Here, we add the ```ResultsRepeater(DummyUpdateGenerator(), n=num_fast_images)``` block to the callback to immediately add a pre-set number of tasks to the queue. Optionally, one could re-add a fast image after the previous like in the slow images (with a check to stop once the event is over).

In [ ]:
from autosted.callback_buildingblocks import NewestSettingsSelector
from autosted.callback_buildingblocks import ResultsRepeater
from autosted.taskgeneration import DummyUpdateGenerator
from autosted.detection import AcceptanceCheck

pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test", hierarchy_levels=["slow", "fast"]
)

next_slow_image_callback = AcquisitionTaskGenerator(
    "slow", JSONSettingsLoader(parameters), delay=10
)

# simple boolean check if the index of latest "slow" acquisition is divisible by switch frequency
switch_after_n_slow_images = 3
# NOTE: the check gets passed an image, but we just ignore it
# in a realistic application, we would detect an event of interest in the image
check_function = (
    lambda img: (max(pipeline.data.keys())[0] + 1) % switch_after_n_slow_images == 0
)

num_fast_images = 3
fast_images_callback = AcquisitionTaskGenerator(
    "fast",
    # 1. check if we should switch to fast mode
    # if True, this returns a dummy update, if False, no updates at all
    AcceptanceCheck(check_function),
    # 2. get actual settings (copy "slow" settings)
    NewestSettingsSelector(),
    # 3. repeat a dummy update so we actually enqueue multiple "fast" images
    ResultsRepeater(DummyUpdateGenerator(), n=num_fast_images),
    delay=5,  # time to wait from last acquisition at level "fast"
)

pipeline.add_callback(next_slow_image_callback, "slow")
pipeline.add_callback(fast_images_callback, "slow")
pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(20))

pipeline.run(next_slow_image_callback)